# Chords Progressions Transformer Training Dataset Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone --depth 1 https://github.com/asigalov61/tegridy-tools

In [ ]:
#@title Import all needed modules

print('Loading core modules. Please wait...')

import os
import copy
import math
import statistics
import random

from joblib import Parallel, delayed, parallel_config

from collections import Counter

from tqdm import tqdm

from google.colab import files

print('Creating IO dirs...')

if not os.path.exists('/content/Dataset'):
  os.makedirs('/content/Dataset')

if not os.path.exists('/content/INTS'):
  os.makedirs('/content/INTS')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD MIDI DATASET)

In [ ]:
# @title Download and untar LAKH clean_midi MIDI subset
%cd /content/Dataset/
!wget https://huggingface.co/datasets/projectlosangeles/Monster-MIDI-Dataset/resolve/main/Monster_MIDI_Dataset_Search_Results_Ver_1_0_CC_BY_NC_SA.zip
!unzip Monster_MIDI_Dataset_Search_Results_Ver_1_0_CC_BY_NC_SA.zip
!rm Monster_MIDI_Dataset_Search_Results_Ver_1_0_CC_BY_NC_SA.zip
%cd /content/

# (FILE LIST)

In [ ]:
#@title Save file list
###########

print('=' * 70)
print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"

filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    for file in filenames:
        if file.endswith(('.mid', '.midi', '.kar')):
            filez.append(os.path.join(dirpath, file))
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

else:
  print('Randomizing file list...')
  random.shuffle(filez)
  print('=' * 70)

  TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/filez')
  print('=' * 70)

In [ ]:
#@title Load file list

print('=' * 70)
filez = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/filez')
print('Done!')
print('=' * 70)

# (PROCESS)

In [ ]:
# @title Load TMIDIX MIDI Processor

print('=' * 70)
print('Loading TMIDIX MIDI Processor...')

def TMIDIX_MIDI_Processor(midi_file):

    try:

        fn = os.path.basename(midi_file)

        #=======================================================
        # START PROCESSING

        #===============================================================================
        # Raw single-track ms score

        raw_score = TMIDIX.midi2single_track_ms_score(midi_file)

        #===============================================================================
        # Enhanced score notes

        escore_notes = TMIDIX.advanced_score_processor(raw_score, return_enhanced_score_notes=True)[0]

        no_drums_escore_notes = [e for e in escore_notes if e[6] < 80]

        if len(no_drums_escore_notes) > 0:

            #=======================================================
            # PRE-PROCESSING

            #===============================================================================
            # Augmented enhanced score notes

            no_drums_escore_notes = TMIDIX.augment_enhanced_score_notes(no_drums_escore_notes)

            cscore = TMIDIX.chordify_score([1000, no_drums_escore_notes])

            clean_cscore = []

            for c in cscore:
                pitches = []
                cho = []
                for cc in c:
                    if cc[4] not in pitches:
                        cho.append(cc)
                        pitches.append(cc[4])

                clean_cscore.append(cho)

            #=======================================================
            # FINAL PROCESSING

            melody_chords = []

            #=======================================================
            # MAIN PROCESSING CYCLE
            #=======================================================

            pe = clean_cscore[0][0]

            first_chord = True

            for c in clean_cscore:

                # Chords

                c.sort(key=lambda x: x[4], reverse=True)

                tones_chord = sorted(set([cc[4] % 12 for cc in c]))

                try:
                    chord_token = TMIDIX.ALL_CHORDS_SORTED.index(tones_chord)
                except:
                    checked_tones_chord = TMIDIX.check_and_fix_tones_chord(tones_chord)
                    chord_token = TMIDIX.ALL_CHORDS_SORTED.index(checked_tones_chord)

                melody_chords.extend([chord_token+384])

                if first_chord:
                        melody_chords.extend([0])
                        first_chord = False

                for e in c:

                    #=======================================================
                    # Timings...

                    time = e[1]-pe[1]

                    dur = e[2]

                    if time != 0 and time % 2 != 0:
                        time += 1
                    if dur % 2 != 0:
                        dur += 1

                    delta_time = int(max(0, min(255, time)) / 2)

                    # Durations

                    dur = int(max(0, min(255, dur)) / 2)

                    # Pitches

                    ptc = max(1, min(127, e[4]))

                    #=======================================================
                    # FINAL NOTE SEQ

                    # Writing final note asynchronously

                    if delta_time != 0:
                        melody_chords.extend([delta_time, dur+128, ptc+256])
                    else:
                        melody_chords.extend([dur+128, ptc+256])

                    pe = e

                if len(melody_chords) > 8192:
                    break

            #=======================================================

            # TOTAL DICTIONARY SIZE 706+1=707
            #=======================================================

            return melody_chords[:8193]

        else:
            return None

    except Exception as e:
        print('=' * 70)
        print('ERROR!!!')
        print('File name:', midi_file)
        print('Error:', e)
        print('=' * 70)
        return None

print('Done!')
print('=' * 70)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

NUMBER_OF_PARALLEL_JOBS = 16 # Number of parallel jobs
NUMBER_OF_FILES_PER_ITERATION = 16 # Number of files to queue for each parallel iteration
SAVE_EVERY_NUMBER_OF_ITERATIONS = 160 # Save every 2560 files

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

melody_chords_f = []

files_count = 0
good_files_count = 0

print('Processing MIDI files. Please wait...')
print('=' * 70)

for i in tqdm(range(0, len(filez), NUMBER_OF_FILES_PER_ITERATION)):

  with parallel_config(backend='threading', n_jobs=NUMBER_OF_PARALLEL_JOBS, verbose = 0):
    output = Parallel(n_jobs=NUMBER_OF_PARALLEL_JOBS, verbose=0)(delayed(TMIDIX_MIDI_Processor)(f) for f in filez[i:i+NUMBER_OF_FILES_PER_ITERATION])

  for o in output:

      if o is not None:
          melody_chords_f.append(o)

  # Saving every 2560 processed files
  if i % (SAVE_EVERY_NUMBER_OF_ITERATIONS * NUMBER_OF_FILES_PER_ITERATION) == 0 and i != 0:
      print('SAVING !!!')
      print('=' * 70)
      good_files_count += len(melody_chords_f)
      print('Saving processed files...')
      print('=' * 70)
      print('Data check:', min(melody_chords_f[0]), '===', max(melody_chords_f[0]), '===', len(list(set(melody_chords_f[0]))), '===', len(melody_chords_f[0]))
      print('=' * 70)
      print('Processed so far:', good_files_count, 'out of', i, '===', good_files_count / i, 'good files ratio')
      print('=' * 70)
      count = str(i)
      TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/INTS/CPT_INTs_'+count)
      melody_chords_f = []
      print('=' * 70)

print('SAVING !!!')
print('=' * 70)
good_files_count += len(melody_chords_f)
print('Saving processed files...')
print('=' * 70)
print('Data check:', min(melody_chords_f[0]), '===', max(melody_chords_f[0]), '===', len(list(set(melody_chords_f[0]))), '===', len(melody_chords_f[0]))
print('=' * 70)
print('Processed so far:', good_files_count, 'out of', i, '===', good_files_count / i, 'good files ratio')
print('=' * 70)
count = str(i)
TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/INTS/CPT_INTs_'+count)
print('=' * 70)

# (TEST INTS)

In [ ]:
#@title Test INTs

train_data1 = random.choice(melody_chords_f)

print('=' * 70)
print('Seq len:', len(train_data1))
print('Sample INTs', train_data1[:15])
print('=' * 70)

out = train_data1

if len(out) != 0:

    song = out
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    patches = [0] * 16

    for ss in song:

        if 0 <= ss < 128:

            time += ss

        if 128 <= ss < 256:

            dur = (ss-128)

        if 256 <= ss < 384:

            pitch = (ss-256)

            song_f.append(['note', time, dur, channel, pitch, vel ])

detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Chords Progressions Transformer',
                                                        output_file_name = '/content/Chords-Progressions-Transformer-Composition',
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=patches,
                                                        timings_multiplier=32
                                                        )
print('=' * 70)

# (ZIP AND DOWNLOAD INTS)

In [ ]:
#@title Zip and download training INTs

print('=' * 70)

try:
    os.remove('Chords_progressions_Transformer_INTs.zip')
except OSError:
    pass

print('Zipping... Please wait...')
print('=' * 70)

%cd /content/INTS/
!zip Chords_progressions_Transformer_INTs.zip *.pickle
%cd /content/

print('=' * 70)
print('Done!')
print('=' * 70)

print('Downloading final zip file...')
print('=' * 70)

files.download('/content/INTS/Chords_progressions_Transformer_INTs.zip')

print('Done!')
print('=' * 70)

# Congrats! You did it! :)